**Project 9: Heating and cooling power DFAB**

Course: EDE WS 2025-2026 Semester Project

Date: 15.01.2025

Team Members: Aya Abd Alahi, John Riecken

**1. Introduction**

Briefly describe the dataset and the problem you are solving. What is the target variable? Why is this prediction useful?

In [2]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

In [3]:
# import data

df = pd.read_csv('../data/dfab_2019_2022_resampled.csv')
df.head()

,Unnamed: 0,time,heating_cooling_power,irrad,kitchen_active_power,setp_371,setp_472,setp_474,setp_476,setp_571,...,y2_574,y3_371,y3_472,y3_476,y3_571,y3_574,y4_371,y5_371,y6_371,y7_371
0,0,2019-07-01 00:00:00,-3.330995,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,1.0,1.0,1.0,1.0,1.0,0.400000,1.0,1.0,1.0,1.0
1,1,2019-07-01 00:15:00,-3.087947,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,1.0,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,1.0
2,2,2019-07-01 00:30:00,-3.441945,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,1.0,1.0,1.0,1.0,1.0,0.600000,1.0,1.0,1.0,1.0
3,3,2019-07-01 00:45:00,-3.274495,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,1.0,1.0,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0
4,4,2019-07-01 01:00:00,-3.298245,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,1.0,1.0,1.0,1.0,1.0,0.666667,1.0,1.0,1.0,1.0


In [6]:
# check metadata
# used AI for encoding

meta_df = pd.read_csv('../data/dfab_metadata.csv', encoding='iso-8859-1')
meta_df

,NumericID,DescriptiveID,Description,Unit,Type
0,Index,Index,Index,Index,Index
1,3200000,temp_amb,Ambient temperature [°C],roof,Temperature Ambient
2,3200008,irrad,Global solar irradiation [W/m^2],roof,Solar Radiation
3,421110008,y1_371,Valve 1 position room 371 [0/1],dfab,Valve
4,421110009,y2_371,Valve 2 position room 371 [0/1],dfab,Valve
5,421110010,y3_371,Valve 3 position room 371 [0/1],dfab,Valve
6,421110011,y4_371,Valve 4 position room 371 [0/1],dfab,Valve
7,421110012,y5_371,Valve 5 position room 371 [0/1],dfab,Valve
8,421110013,y6_371,Valve 6 position room 371 [0/1],dfab,Valve
9,421110014,y7_371,Valve 7 position room 371 [0/1],dfab,Valve


In [10]:
# drop excluded variables
to_drop = 'y1_371 y1_472 y1_474 y1_476 y1_571 y1_573 y1_574 y2_371 y2_472 y2_476 y2_571 y2_574 y3_371 y3_472 y3_476 y3_571 y3_574 y4_371 y5_371 y6_371 y7_371'.split()
df.drop(columns=to_drop, inplace=True)
df.head()

,Unnamed: 0,time,heating_cooling_power,irrad,kitchen_active_power,setp_371,setp_472,setp_474,setp_476,setp_571,...,shower_571,temp_371,temp_472,temp_474,temp_476,temp_571,temp_573,temp_574,temp_amb,total_active_power
0,0,2019-07-01 00:00:00,-3.330995,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,0.0,27.500000,26.5,28.0,29.0,31.0,30.500000,31.5,25.926667,0.575526
1,1,2019-07-01 00:15:00,-3.087947,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,0.0,27.500000,26.5,28.0,29.0,30.8,30.500000,31.5,25.586667,0.689346
2,2,2019-07-01 00:30:00,-3.441945,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,0.0,27.500000,26.5,28.0,29.0,30.5,30.500000,31.5,25.373333,0.675612
3,3,2019-07-01 00:45:00,-3.274495,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,0.0,27.500000,26.5,28.0,29.0,30.5,30.466667,31.5,25.333333,0.566832
4,4,2019-07-01 01:00:00,-3.298245,0.0,0.0,25.0,22.0,22.0,19.0,25.5,...,0.0,27.333333,26.5,28.0,29.0,30.5,30.000000,31.2,24.900000,0.481446


In [11]:
df.shape

(140256, 23)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140256 entries, 0 to 140255
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             140256 non-null  int64  
 1   time                   140256 non-null  object 
 2   heating_cooling_power  138446 non-null  float64
 3   irrad                  137900 non-null  float64
 4   kitchen_active_power   138625 non-null  float64
 5   setp_371               132951 non-null  float64
 6   setp_472               133554 non-null  float64
 7   setp_474               132949 non-null  float64
 8   setp_476               132360 non-null  float64
 9   setp_571               132951 non-null  float64
 10  setp_573               132675 non-null  float64
 11  setp_574               132663 non-null  float64
 12  shower_471             138776 non-null  float64
 13  shower_571             137960 non-null  float64
 14  temp_371               137795 non-nu

**2. Part 1: Exploratory Data Analysis (EDA)**

**2.1 Statistical Summary**

Show basic stats (mean, median, std dev) and handle any missing values.

In [17]:
df.describe()

,Unnamed: 0,heating_cooling_power,irrad,kitchen_active_power,setp_371,setp_472,setp_474,setp_476,setp_571,setp_573,...,shower_571,temp_371,temp_472,temp_474,temp_476,temp_571,temp_573,temp_574,temp_amb,total_active_power
count,140256.000000,138446.000000,137900.000000,138625.000000,132951.000000,133554.000000,132949.000000,132360.000000,132951.000000,132675.000000,...,137960.000000,137795.000000,137710.000000,137450.000000,136886.000000,137238.000000,137421.000000,137361.000000,139273.000000,138936.000000
mean,17531.524641,0.157416,184.621133,0.041175,22.138905,21.873563,21.906849,20.431112,22.686919,22.109252,...,0.004772,23.571796,23.276319,23.382233,23.976069,23.352456,23.736727,24.169731,12.073460,-0.105771
std,10122.183718,1.833801,303.647495,0.211394,2.403091,1.889170,1.327516,2.585235,2.681995,2.062852,...,0.111677,2.109775,1.729603,2.214518,2.666358,2.177219,2.403533,2.765282,8.459903,1.437322
min,0.000000,-8.420386,0.000000,0.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,0.000000,18.500000,17.500000,16.000000,19.000000,18.500000,19.000000,19.000000,-9.940000,-6.151530
25%,8765.750000,-1.113597,0.000000,0.000000,22.000000,22.000000,22.000000,19.000000,22.000000,22.000000,...,0.000000,22.000000,22.000000,21.500000,22.000000,21.766667,22.000000,22.000000,5.306667,-0.376740
50%,17531.500000,0.000000,11.266667,0.000000,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000,...,0.000000,23.000000,23.000000,22.500000,23.000000,22.500000,23.000000,23.000000,11.566667,0.443226
75%,26297.250000,1.496797,249.133333,0.000000,23.500000,23.000000,22.000000,22.000000,24.500000,22.000000,...,0.000000,25.000000,24.000000,25.000000,26.000000,24.500000,25.500000,26.000000,18.033333,0.644574
max,35135.000000,6.418994,1250.000000,5.157067,28.000000,28.000000,28.000000,28.000000,30.000000,28.500000,...,7.921111,46.000000,47.000000,44.000000,44.000000,46.000000,44.000000,45.000000,40.120000,7.180404


**2.2 Visual Exploration**

Focussed plots (Histograms, Boxplots) that reveal insights about the data distribution.

**2.3 Correlation Analysis**

Heatmaps or scatter plots showing the relationship between features and the target variable.

**3. Part 2: Linear Regression Modeling**

**3.1 Data Preprocessing**

Train-test splits, scaling, or encoding categorical variables.

**3.2 Model Iterations (The 5 Models)**

List and train your five variations here (e.g., Simple Linear, Multi-feature, Polynomial, Lasso/Ridge).

**3.3 Feature Engineering & Selection**

Explain the systematic approach used to find the best feature set (e.g., RFE or SelectKBest).

**4. Evaluation & Model Selection**

4.1 Cross-Validation Results
Use k-fold cross-validation to show the stability of your models.

**4.2 Visual Comparison**

Plotting Predicted vs. Actual values or Residual plots for all models.

**4.3 Final Model DescriptionWhy did you pick this model?**

Discuss its structure ($R^2$, RMSE) and how it balances bias and variance.

**5. Appendix**

**5.1 Contribution Matrix**


**5.2 Sources & AI Transparency**

Libraries: NumPy, Pandas, Scikit-Learn, Matplotlib.

External Data: [Link to source].

AI Usage Disclosure: * LLM used for: Debugging the cross-validation loop and generating the Markdown table structures.

Human intervention: All statistical interpretations and final model selection decisions were made by the team.